In [12]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AdamW

import torch
import pandas as pd
import numpy as np

In [13]:
# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
class SarcasmDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [15]:
data = pd.read_csv("../data/train-balanced-sarcasm.csv")

# remove na
data.dropna(subset=['comment'], inplace = True)

data.reset_index(drop=True, inplace=True)

dataset = SarcasmDataset(data['comment'], data['label'], tokenizer)

In [16]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

In [17]:
if torch.backends.mps.is_available():  # for Mac M1/M2/M3
    device = torch.device("mps")
elif torch.cuda.is_available(): # for CUDA-enabled GPUs
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
print(f"Using {device} device")

Using cuda device


In [18]:
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
for epoch in range(3):  
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

c:\Users\zhang\anaconda3\envs\DL\lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
100%|██████████| 101077/101077 [24:58:34<00:00,  1.12it/s]         


Epoch 1, Loss: 51490.7062


100%|██████████| 101077/101077 [3:56:12<00:00,  7.13it/s] 


Epoch 2, Loss: 46150.7656


100%|██████████| 101077/101077 [4:01:26<00:00,  6.98it/s] 

Epoch 3, Loss: 42104.7787


In [21]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())



In [22]:
# Accuracy
acc = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {acc:.4f}")

# Precision, Recall, F1-score (macro, micro, weighted, etc.)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

# Full classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds))

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

Accuracy: 0.7569
Precision: 0.7571, Recall: 0.7569, F1-score: 0.7569

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.77      0.76    100879
           1       0.76      0.74      0.75    101276

    accuracy                           0.76    202155
   macro avg       0.76      0.76      0.76    202155
weighted avg       0.76      0.76      0.76    202155

Confusion Matrix:
[[77752 23127]
 [26016 75260]]


In [20]:
model.save_pretrained("../model/")
tokenizer.save_pretrained("../model/")

('../model/tokenizer_config.json',
 '../model/special_tokens_map.json',
 '../model/vocab.txt',
 '../model/added_tokens.json')